# The Cuppa3 Compiler

This is a notebook that let's us play around with the Cuppa3 compiler. Both as a whole running our testsuite and the individual parts.

In [1]:
from cuppa3_cc import cc
from exp2bytecode_interp import interp as run

## The test suite.

In [2]:
add = \
'''
declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;
'''

seqsum = \
'''
declare add(a,b) 
{
    return a+b;
}

declare seqsum(n) 
{
    
    declare i = 1;
    declare sum = 0;
        
    while (i <= n) 
    {
        sum = add(sum,i);
        i = i + 1;
    }
        
    put sum;
}

seqsum(10);
'''

and_prog = \
'''
declare and(a,b)
{
    return a*b;
}

put and(1,1);
put and(1,0);
put and(0,1);
put and(0,0);
'''

fact = \
'''
// computes the factorial of x = 3
declare x = 3;
declare y = 1;
while (1 <= x) 
{
      y = y * x;
      x = x - 1;
}
put y;
'''

factrec = \
'''
// recursive implementation of factorial
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}

put fact(3);
'''

fold = \
'''
declare x = (3 + 2) / 5;
put x;
'''


func1 = \
'''
declare f () {
 put(1001);
}

f();
'''

In [3]:
print(add)


declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;



In [10]:
print(cc(add))

	jump L10 ;
#  
# Start of function add
#  
add:
	pushf 3 ;
	store %tsx[0] %tsx[-4] ;
	store %tsx[-1] %tsx[-5] ;
	store %tsx[-2] (+ %tsx[0] %tsx[-1]) ;
	store %rvx %tsx[-2] ;
	popf 3 ;
	return ;
	popf 3 ;
	return ;
#  
# End of function add
#  
L10:
	noop ;
	pushv 2 ;
	pushv 3 ;
	call add ;
	popv ;
	popv ;
	store t$0 %rvx ;
	store t$1 t$0 ;
	print t$1 ;
	stop ;



In [5]:
print(seqsum)


declare add(a,b) 
{
    return a+b;
}

declare seqsum(n) 
{
    
    declare i = 1;
    declare sum = 0;
        
    while (i <= n) 
    {
        sum = add(sum,i);
        i = i + 1;
    }
        
    put sum;
}

seqsum(10);



In [9]:
run(cc(seqsum))

> 20


In [7]:
print(and_prog)


declare and(a,b)
{
    return a*b;
}

put and(1,1);
put and(1,0);
put and(0,1);
put and(0,0);



In [8]:
run(cc(and_prog))

> 1
> 0
> 0
> 0


In [9]:
print(fact)


// computes the factorial of x = 3
declare x = 3;
declare y = 1;
while (1 <= x) 
{
      y = y * x;
      x = x - 1;
}
put y;



In [10]:
run(cc(fact))

> 6


In [11]:
print(factrec)


// recursive implementation of factorial
declare fact(x) 
{
     if (x <= 1)
        return 1;
     else 
        return fact(x-1) * x;
}

put fact(3);



In [14]:
run(cc(factrec))

> 6


In [13]:
print(fold)


declare x = (3 + 2) / 5;
put x;



In [14]:
run(cc(fold))

> 1


In [15]:
print(func1)


declare f () {
 put(1001);
}

f();



In [16]:
run(cc(func1))

> 1001


Testing our interpreter on the recursive computation of the Fibonacci sequence

In [17]:
fib = \
'''
declare fib(n)
{    
    if (n == 0)
        return 0;
    else if (n == 1)
        return 1;
    else
        return fib(n-1) + fib(n-2);
}

// fibonacci sequence: 0,1,1,2,3,5,8,13,21,34,55,89,144,...
declare i = 0
while (i <= 12)
{
    put fib(i)
    i = i + 1
}
'''

In [18]:
print(cc(fib))

	jump L12 ;
#  
# Start of function fib
#  
fib:
	pushf 8 ;
	store %tsx[0] %tsx[-9] ;
	store %tsx[-1] (== %tsx[0] 0) ;
	jumpF %tsx[-1] L13 ;
	store %rvx 0 ;
	popf 8 ;
	return ;
	jump L14 ;
L13:
	store %tsx[-2] (== %tsx[0] 1) ;
	jumpF %tsx[-2] L15 ;
	store %rvx 1 ;
	popf 8 ;
	return ;
	jump L16 ;
L15:
	store %tsx[-3] (- %tsx[0] 1) ;
	pushv %tsx[-3] ;
	call fib ;
	popv ;
	store %tsx[-4] %rvx ;
	store %tsx[-5] (- %tsx[0] 2) ;
	pushv %tsx[-5] ;
	call fib ;
	popv ;
	store %tsx[-6] %rvx ;
	store %tsx[-7] (+ %tsx[-4] %tsx[-6]) ;
	store %rvx %tsx[-7] ;
	popf 8 ;
	return ;
L16:
	noop ;
L14:
	noop ;
	popf 8 ;
	return ;
#  
# End of function fib
#  
L12:
	noop ;
	store t$0 0 ;
L17:
	store t$1 (<= t$0 12) ;
	jumpF t$1 L18 ;
	pushv t$0 ;
	call fib ;
	popv ;
	store t$2 %rvx ;
	print t$2 ;
	store t$3 (+ t$0 1) ;
	store t$0 t$3 ;
	jump L17 ;
L18:
	noop ;
	stop ;



## Testing parts of the compiler

In [5]:
from cuppa3_lex import lexer
from cuppa3_cc_frontend_gram import parser
from cuppa3_cc_state import state
from cuppa3_cc_tree_rewrite import walk as rewrite
from cuppa3_cc_codegen import walk as codegen
from cuppa3_cc_output import output
from grammar_stuff import dump_AST
from cuppa3_cc import cc
from pprint import pprint

In [6]:
state.initialize()

In [7]:
print(add)


declare add(a,b) 
{
    return a+b;
}

declare x = add(3,2);
put x;



In [8]:
parser.parse(add, lexer=lexer)

In [9]:
dump_AST(state.AST)


(seq 
  |(fundecl add 
  |  |(seq 
  |  |  |(id a) 
  |  |  |(seq 
  |  |  |  |(id b) 
  |  |  |  |(nil))) 
  |  |(block 
  |  |  |(seq 
  |  |  |  |(return 
  |  |  |  |  |(+ 
  |  |  |  |  |  |(id a) 
  |  |  |  |  |  |(id b))) 
  |  |  |  |(nil)))) 
  |(seq 
  |  |(declare x 
  |  |  |(callexp add 
  |  |  |  |(seq 
  |  |  |  |  |(integer 3) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |(nil))))) 
  |  |(seq 
  |  |  |(put 
  |  |  |  |(id x)) 
  |  |  |(nil))))


In [10]:
t = rewrite(state.AST)
dump_AST(t)


(seq 
  |(fundef add 
  |  |(seq %tsx[0] 
  |  |  |(seq %tsx[-1] 
  |  |  |  |(nil))) 
  |  |(block 
  |  |  |(seq 
  |  |  |  |(return 
  |  |  |  |  |(+ %tsx[-2] 
  |  |  |  |  |  |(id %tsx[0]) 
  |  |  |  |  |  |(id %tsx[-1]))) 
  |  |  |  |(nil))) 3) 
  |(seq 
  |  |(assign t$1 
  |  |  |(callexp t$0 add 
  |  |  |  |(seq 
  |  |  |  |  |(integer 3) 
  |  |  |  |  |(seq 
  |  |  |  |  |  |(integer 2) 
  |  |  |  |  |  |(nil))))) 
  |  |(seq 
  |  |  |(put 
  |  |  |  |(id t$1)) 
  |  |  |(nil))))


In [11]:
print(output(codegen(t)))

	jump L0 ;
#  
# Start of function add
#  
add:
	pushf 3 ;
	store %tsx[0] %tsx[-4] ;
	store %tsx[-1] %tsx[-5] ;
	store %tsx[-2] (+ %tsx[0] %tsx[-1]) ;
	store %rvx %tsx[-2] ;
	popf 3 ;
	return ;
	popf 3 ;
	return ;
#  
# End of function add
#  
L0:
	noop ;
	pushv 2 ;
	pushv 3 ;
	call add ;
	popv ;
	popv ;
	store t$0 %rvx ;
	store t$1 t$0 ;
	print t$1 ;



## Scratch Area

In [26]:
prog_global = \
'''
declare x = 1;
{
        declare x = 2;
        put x;
}
{
        declare x = 3;
        put x;
}
put x;
'''

In [27]:
print(cc(prog_global))

	store t$0 1 ;
	store t$1 2 ;
	print t$1 ;
	store t$2 3 ;
	print t$2 ;
	print t$0 ;
	stop ;



In [28]:
print(cc('put 3*2+4;'))

	store t$0 (* 3 2) ;
	store t$1 (+ t$0 4) ;
	print t$1 ;
	stop ;



In [29]:
call_in_exp = \
'''
declare inc(k) return k+1;

put 3*2+inc(5);
'''
print(cc(call_in_exp))

	jump L20 ;
#  
# Start of function inc
#  
inc:
	pushf 2 ;
	store %tsx[0] %tsx[-3] ;
	store %tsx[-1] (+ %tsx[0] 1) ;
	store %rvx %tsx[-1] ;
	popf 2 ;
	return ;
	popf 2 ;
	return ;
#  
# End of function inc
#  
L20:
	noop ;
	store t$0 (* 3 2) ;
	pushv 5 ;
	call inc ;
	popv ;
	store t$1 %rvx ;
	store t$2 (+ t$0 t$1) ;
	print t$2 ;
	stop ;

